In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_customer = read_file(spark,"parquet",f'{bronze}/customers')

df_cust_rmd = deduplicate(df_customer,["customer_id"])

rules = {"customer_city":"lower_trim",
         "customer_state":"upper_trim"}

df_standard = standardize_string_columns(df_cust_rmd, rules)

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "customers"}}

df_final_cust = enrich_add_col(df_standard, columns)
# df_final_cust.display()

In [0]:
# Write data delta file /delta table in silver location

write_file(df_final_cust,f"{silver}/silver_customers")

write_delta_liquid(df_final_cust, f"{catalog}.{schema}.silver_customers",["customer_id","customer_unique_id"])